In [7]:
import numpy as np
import matplotlib.pyplot as plt
from mp_api.client import MPRester
from pymatgen.io.cif import CifWriter
from pymatgen.core.operations import SymmOp
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.electronic_structure.plotter import BSPlotter
from pymatgen.phonon.plotter import PhononBSPlotter
from jupyter_jsmol.pymatgen import quick_view
from lmapr1492 import plot_brillouin_zone, get_plot_bs, get_plot_dos, get_plot_bs_and_dos, get_branch_wavevectors
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from pymatgen.analysis.diffraction.xrd import XRDCalculator
from pymatgen.ext.matproj import MPRester
from pymatgen.electronic_structure.core import Spin
from scipy.interpolate import UnivariateSpline
import pandas as pd

In [8]:
mp_key = "saKvn4nYOR6knw5q6WoAxtMKCsoAG1gq"
mp_id = "mp-561586"

In [22]:
with MPRester(mp_key) as m:
    prim_struc = m.get_structure_by_material_id(mp_id)
    ph_bs = m.get_phonon_bandstructure_by_material_id(mp_id)
conv_struc = SpacegroupAnalyzer(prim_struc).get_conventional_standard_structure()
symmops = SpacegroupAnalyzer(conv_struc).get_space_group_operations()
ph_plot = PhononBSPlotter(ph_bs)
data = ph_plot.bs_plot_data()
distances = data.get('distances')
frequences = data.get('frequency')

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

INTRODUCTION 

Dans un matériau cristallin, les vibrations atomiques peuvent être décrites par des phonons, qui représentent les quanta d'énergie associés aux ondes élastiques. L'analyse de la structure de bande des phonons permet d'extraire la vitesse du son dans différentes directions de la zone de Brillouin. l'objectif de ce travail est de calculer la vitesse du son pour trois branches acoustiques différentes dans trois directions différentes de la zone de Brillouin.

ZONE DE BRILLOUIN

In [23]:
plot_brillouin_zone(ph_bs.structure)


on remarque bien que le centre de la zone de brillouin correspond à  Γ

STRUCTURE BANDE DE PHONONS

In [24]:
fig_ph_bs = get_plot_bs(ph_bs)
fig_ph_bs.show()

In [64]:
print(f"On calculera la vitesse du son dans la direction \u0393 - X : bande 2")
print(f"On calculera la vitesse du son dans la direction K - \u0393 :  bande 1  ")
print(f"On calculera la vitesse du son dans la direction \u0393 - L :bande 0")

On calculera la vitesse du son dans la direction Γ - X : bande 2
On calculera la vitesse du son dans la direction K - Γ :  bande 1  
On calculera la vitesse du son dans la direction Γ - L :bande 0


CALCUL DES VITESSES DU SON

Calculer la vitesse du son pour trois branches acoustiques différentes dans trois directions différentes de la zone de Brillouin.

La vitesse du son dans un cristal est déterminée à partir de la dispersion phononique en calculant la pente des branches acoustiques près du point Gamma, selon la relation v = dω/dq. Ce calcul repose sur l’hypothèse que, pour de faibles vecteurs d’onde (q → 0), la relation de dispersion est linéaire. On distingue trois branches acoustiques : une longitudinale (LA) et deux transversales (TA1, TA2), dont la vitesse dépend des propriétés élastiques du matériau et de sa direction cristallographique. L’approximation numérique se fait en prenant le rapport des variations Δω / Δq sur les premiers points de la courbe de dispersion. Ce modèle suppose un milieu homogène et anisotrope, avec des interactions harmoniques dominantes entre les atomes.

In [48]:
def compute_speed_of_sound(q_values, f_values):
    q_values = np.array(q_values) * 1e10  # Conversion Å^-1 en m^-1
    f_values = np.array(f_values) * 1e12  # Conversion THz en Hz
    
    dq = q_values[1] - q_values[0]
    dw = 2 * np.pi * (f_values[1] - f_values[0])
    result = np.abs(dw / dq)
    return result


In [49]:

v_s1 = compute_speed_of_sound(distances[4], frequences[4][2])
v_s2 = compute_speed_of_sound(distances[1], frequences[1][1])
v_s3 = compute_speed_of_sound(distances[0], frequences[0][0])

print(f"Vitesse du son dans la direction \u0393 - X : {v_s1:.2f} m/s")
print(f"Vitesse du son dans la direction K - \u0393 : {v_s2:.2f} m/s")
print(f"Vitesse du son dans la direction \u0393 - L : {v_s3:.2f} m/s")

Vitesse du son dans la direction Γ - X : 1884.96 m/s
Vitesse du son dans la direction K - Γ : 1256.64 m/s
Vitesse du son dans la direction Γ - L : 628.32 m/s
